In this notebook we want to generate a csv file with labels that represent what we want to do with every picture for preprocessing.

In [117]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import defaultdict
import tqdm
print(os.listdir("."))
import csv

['labels.csv', '.DS_Store', 'ZIP-files', 'Images', 'README.md', 'generate_counts.py', '.gitignore', 'random_crops.py', 'download_images.py', '.ipynb_checkpoints', 'data_viz.ipynb', '.git', 'CSV-files', 'rotate.py', 'Images-test', 'generate_labels.py.ipynb']


id: unique id of image 

url: download url

landmark_id: unique landmark id

In [118]:
train_data = pd.read_csv('./CSV-files/train.csv')
train_data.head(5)

,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231


In [132]:
print('num images: ', train_data.shape[0])

num images:  1225029


In [137]:
# Maps landmark_id to number of images with that label
counts = train_data['landmark_id'].value_counts()
counts.head(5)

9633    50337
6051    50148
6599    23415
9779    18471
2061    13271
Name: landmark_id, dtype: int64

In [121]:
# Maps landmark_id to id 
lid_to_img = defaultdict(list)
for row in train_data.iterrows():
    lid = row[1]['landmark_id']
    img_id = row[1]['id']
    lid_to_img[lid].append(img_id)

In [133]:
print('num_labels: ', len(lid_to_img))

num_labels:  14951


In [122]:
# Sanity check: should be 50337
len(lid_to_img[9633])

50337

In [134]:
labels = {} # Maps l_id to 
for l_id, l_id_count in counts.iteritems():
    if l_id_count < 5:
        label = 'upsample_1' # flip (x2), dim/bright (x3), color-transform (x2), crop (x5), total: x60, downsample to 60
    elif l_id_count < 15:
        label = 'upsample_2' # flip (x2), dim/bright (x3), crop (x2), total: x12, downsample to 60
    elif l_id_count < 30:
        label = 'upsample_3' # flip (x2), crop (x2), total: x4, downsample to 60
    elif l_id_count < 60:
        label = 'upsample_4' # rotate (x2), downsample to 60
    else:
        label = 'downsample' # take 60 random images for each directory
    labels[l_id] = label
        

In [138]:
# sanity check
print('counts: ', counts[9633])
print(labels[9633])

counts:  50337
downsample


In [129]:
with open('labels.csv', 'w') as csvfile:
    fieldnames = ['id', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for k in labels:   
        writer.writerow({'id': k, 'label': labels[k]})